In [1]:
import pandas as pd
import numpy as np
import datetime as datetime
import matplotlib.pyplot as plt
hsNData1 = pd.read_csv(r'E:\Dhaval\Elastic Search\Python\E-Book\Data File\highstoragesystem-data-for-energy-optimization\HRSS_normal_optimized.csv')

In [2]:
#def clean_strings(strings):
result = []
cnt = 0
for index, value in enumerate(hsNData1['Timestamp']):
    if index == 0:
        result.append(cnt)
    elif index == len(hsNData1['Timestamp'])-1:
        result.append(cnt)
    elif hsNData1['Timestamp'][index] - hsNData1['Timestamp'][index - 1] < 0:        
        result.append(cnt+1)
        cnt += 1
    else:
        result.append(cnt)

In [3]:
hsNData1['sampleNum'] = result

In [4]:
def resampleDataperSample(Sampledata, noOfSamples, noOfTimeStamps, attributes):  
    data_sampled=np.zeros([noOfSamples, noOfTimeStamps, attributes]) # strictly speaking w/o the first 3 columns! only the 18 attributes
    time_steps= np.arange(noOfTimeStamps)*0.05 
    time_steps
    data_org= Sampledata
    time_index = np.zeros([noOfTimeStamps,])
    time_lower = np.zeros([noOfTimeStamps,])
    time_upper = np.zeros([noOfTimeStamps,])
    alpha = np.zeros([noOfTimeStamps,])
    for i in range(noOfTimeStamps):
        time_index[i] = np.sum(((np.array(data_org['Timestamp'])- time_steps[i])<=0)) -1 
    time_index.astype(int)
    time_lower = np.array(data_org['Timestamp'].loc[time_index]) #there should be no nans
    time_upper = np.array(data_org['Timestamp'].loc[time_index+1]) # last may contain nans, to be catched later
    alpha = (time_upper - time_steps)/(time_upper-time_lower)
    data_values=np.array(data_org)

    # rewrite in matrix form, will be faster
    for i in range(noOfTimeStamps):
        for j in range(attributes):
            tt=(time_index[i]+1).astype(int)
            if (time_index[i]+1)>len(data_org)-1:
                tt=len(data_org)-1
            data_sampled[1,i,j]=alpha[i]*data_values[time_index[i].astype(int),j] + (1-alpha[i])*data_values[tt,j]
    return pd.DataFrame(data_sampled[1,:,:])
    # finally catch NaNs, set to zero
    # interate through samples
    # noOfSamples no. of samples, noOfTimeStamps no. of time stamps and attributes no. of attributes
    # len(data_org)-1 no of time stamps in original data/ no of rows per sample

In [5]:
_noOfSamples = max(result)
_attributes = len(hsNData1.columns)
_noOfTimeStamps = 241
df6 = pd.DataFrame(columns=np.arange(len(hsNData1.columns)))
for i in range(_noOfSamples+1):
    data = hsNData1.loc[hsNData1['sampleNum'] == i]
    data.index = np.arange(len(data))
    df3 = resampleDataperSample(data, _noOfSamples, _noOfTimeStamps, _attributes)
    df6 = df6.append(df3, ignore_index=False, verify_integrity=False, sort=None)
df6.index=np.arange(len(df6))
df7 = df6.dropna(axis=0, how='all')
df7.to_csv(r'E:\Dhaval\Elastic Search\Python\E-Book\Data File\highstoragesystem-data-for-energy-optimization\ResampledData.csv', sep=',')

C:\Users\Snehal.Waman\python\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  
